This is based off of https://github.com/superlinked/chunking-research/blob/main/preprocess.py

In [55]:
import datasets
from datasets import load_dataset

dataset = load_dataset("hotpotqa/hotpot_qa", "fullwiki")
dataset

/Users/aidand/dev/hello-retrieval-eval/env/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for hotpotqa/hotpot_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hotpotqa/hotpot_qa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 90447
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
})

In [56]:
df = dataset['train'].to_pandas()
df.head()

,id,question,answer,type,level,supporting_facts,context
0,5a7a06935542990198eaf050,Which magazine was started first Arthur's Maga...,Arthur's Magazine,comparison,medium,"{'title': ['Arthur's Magazine', 'First for Wom...",{'title': ['Radio City (Indian radio station)'...
1,5a879ab05542996e4f30887e,The Oberoi family is part of a hotel company t...,Delhi,bridge,medium,"{'title': ['Oberoi family', 'The Oberoi Group'...","{'title': ['Ritz-Carlton Jakarta', 'Oberoi fam..."
2,5a8d7341554299441c6b9fe5,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,bridge,hard,"{'title': ['Allie Goertz', 'Allie Goertz', 'Al...","{'title': ['Lisa Simpson', 'Marge Simpson', 'B..."
3,5a82171f5542990a1d231f4a,What nationality was James Henry Miller's wife?,American,bridge,medium,"{'title': ['Peggy Seeger', 'Peggy Seeger', 'Ew...","{'title': ['Moloch: or, This Gentile World', '..."
4,5a84dd955542997b5ce3ff79,Cadmium Chloride is slightly soluble in this c...,alcohol,bridge,medium,"{'title': ['Cadmium chloride', 'Ethanol'], 'se...","{'title': ['Cadmium chloride', 'Water blue', '..."


In [57]:
sample = df.sample(1)
sample['context'].iloc[0]

{'title': array(['Menachem Brenner', 'Kohima Camp', 'High Junk Peak',
        'Clear Water Bay Road', 'Clear Water Bay Peninsula',
        'Clear Water Bay', 'Fat Tong Chau', 'Tai Po Tsai',
        'Hong Kong University of Science and Technology', 'Joss House Bay'],
       dtype=object),
 'sentences': array([array(['Menachem Brenner is a Professor of Finance and a Bank and Financial Analysts Faculty Fellow at New York University Stern School of Business.',
               ' He teaches a course in options and futures, along with an introduction to finance course.',
               ' Brenner also teaches for the Master of Science in Global Finance (MSGF), which is a joint program between Stern and the Hong Kong University of Science and Technology.'],
              dtype=object)                                                                                                                                                                ,
        array(['Kohima Camp or Kohima Barracks as it 

In [58]:
def scentences_all_populated(context):
	for sentences in context['sentences']:
		if not all(sentences):
			return False
	return True

def scentences_populated(context):
	for sentences in context['sentences']:
		if any(sentences):
			return True
	return False

non_true_df = df[~df['context'].apply(scentences_all_populated)]
display(non_true_df.head())
display(non_true_df.shape)
display(non_true_df['context'].sample(1).iloc[0])

,id,question,answer,type,level,supporting_facts,context
474,5ae625335542995703ce8b28,"What football club has won 3 UFEA Super Cups, ...",Liverpool Football Club,bridge,medium,"{'title': ['Hughie Clifford', 'Liverpool F.C.'...","{'title': ['List of FC Barcelona players', 'Su..."
568,5addf5935542995b365fab9e,"Which plant, Buddleja or Burchardia, is an her...",Burchardia,comparison,easy,"{'title': ['Buddleja', 'Burchardia', 'Burchard...","{'title': ['Burchardia', 'Buddleja alata', 'Bu..."
713,5adc05f35542996e685252ac,"Brisingr is the third novel in the ""Inheritanc...",Christopher Paolini,bridge,easy,"{'title': ['Brisingr', 'Inheritance Cycle'], '...","{'title': ['The Wheel of Time', 'Lloyd Alexand..."
857,5a8ba7d75542996e8ac88991,How long was the Underdog show presented by an...,six-week television series,bridge,hard,"{'title': ['The Underdog Show', 'Julian Clary'...","{'title': ['List of Underdog episodes', 'Julia..."
1333,5ae4f18755429960a22e0219,When was the Australian radio and television p...,29 October 1964,bridge,medium,"{'title': ['1 vs. 100 (Australian game show)',...","{'title': ['Lai Van Sam', 'Tim Shaw (salespers..."


(400, 7)

{'title': array(['Socialism in Iran', 'Eastern Bloc',
        'List of diplomatic missions in Nicaragua', 'Third World',
        'The Center for Third World Organizing',
        'New International Economic Order', 'Third World Resurgence',
        'List of diplomatic missions in Cuba',
        'The Third World War: The Untold Story', 'Swedish neutrality'],
       dtype=object),
 'sentences': array([array(['Socialism in Iran or Iranian socialism is a political ideology that traces its beginnings to the 20th century and encompasses various political parties in the country.',
               ' Iran experienced a short Third World Socialism period at the zenith of the Tudeh Party after the abdication of Reza Shah and his replacement by his son, Mohammad Reza Pahlavi (though the party never rose to power).',
               " After failing to reach power, this form of third world socialism was replaced by Mosaddegh's populist, non-aligned Iranian nationalism of the National Front party as the

In [59]:
import numpy as np


df = df.loc[
	df['context'].apply(scentences_populated)
].reset_index(drop=True)
df.head()
# empty_contexts = df[df['context'].apply(lambda x: all(x['sentences']))]
# empty_contexts
# empty_contexts = []
# for i, row in df.iterrows():
# 	if not np.logical_and(row['context']['sentences']):
# 		empty_contexts.append(i)
# empty_contexts

# empty_contexts = df[df['context'].apply(lambda x: all(x['sentences']))]
# empty_contexts



,id,question,answer,type,level,supporting_facts,context
0,5a7a06935542990198eaf050,Which magazine was started first Arthur's Maga...,Arthur's Magazine,comparison,medium,"{'title': ['Arthur's Magazine', 'First for Wom...",{'title': ['Radio City (Indian radio station)'...
1,5a879ab05542996e4f30887e,The Oberoi family is part of a hotel company t...,Delhi,bridge,medium,"{'title': ['Oberoi family', 'The Oberoi Group'...","{'title': ['Ritz-Carlton Jakarta', 'Oberoi fam..."
2,5a8d7341554299441c6b9fe5,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,bridge,hard,"{'title': ['Allie Goertz', 'Allie Goertz', 'Al...","{'title': ['Lisa Simpson', 'Marge Simpson', 'B..."
3,5a82171f5542990a1d231f4a,What nationality was James Henry Miller's wife?,American,bridge,medium,"{'title': ['Peggy Seeger', 'Peggy Seeger', 'Ew...","{'title': ['Moloch: or, This Gentile World', '..."
4,5a84dd955542997b5ce3ff79,Cadmium Chloride is slightly soluble in this c...,alcohol,bridge,medium,"{'title': ['Cadmium chloride', 'Ethanol'], 'se...","{'title': ['Cadmium chloride', 'Water blue', '..."


In [60]:
# df['context'] = df.context.apply(
# 	lambda x: ["".join(sub_list) for sub_list in x['sentences']]
# )
# df.head()


In [61]:

df['context'].sample(1).iloc[0]

{'title': array(['Cold Feet (1989 film)', 'Serguei Kouchnerov', 'A Further Gesture',
        'War and Peace (2007 miniseries)', 'Spartacus (Fast novel)',
        'Spartacus (miniseries)', 'The Venice Project',
        'Requiem for Dominic', 'RFK (film)', 'Robert Dornhelm'],
       dtype=object),
 'sentences': array([array(['Cold Feet is a 1989 film directed by Robert Dornhelm.',
               ' It stars Keith Carradine and Sally Kirkland.'], dtype=object),
        array(['Serguei Kouchnerov (born 22 June 1960 in Rahachow, Belarusian SSR) - artist, animator, director, story artist and screenwriter.',
               ' He started his career as an animator and director in Kiev, Ukraine.',
               ' In 1992, he was hired by Walt Disney Feature Animation as a character animator and came to the United States of America, where he currently lives and works at Illumination Entertainment.'],
              dtype=object)                                                                       

In [ ]:
df['context'] = df.context.apply(
	# They just use the first context
	lambda x: ["".join(sub_list) for sub_list in x['sentences']][0]
)